In [1]:
from google.cloud import bigquery
bq = bigquery.Client()
bigquery.QueryJobConfig(use_query_cache=False)

In [2]:
q = bq.query(
"""
select count(*) as num_rows 
from bigquery-public-data.geo_us_boundaries.counties
""")
q.to_dataframe()

,num_rows
0,3233


In [3]:
#Q1
job_config = bigquery.QueryJobConfig(use_query_cache=False)
q1 = bq.query(
"""
select *
from `bigquery-public-data.geo_us_boundaries.counties`
where lsad_name = "Dane County"
""", job_config = job_config
)
q1.to_dataframe()['geo_id'][0]

'55025'

In [4]:
#Q2
q2 = bq.query(
"""
select state_fips_code, count(state_fips_code) as COUNT
from `bigquery-public-data.geo_us_boundaries.counties`
group by state_fips_code
order by COUNT desc
limit 5
""", job_config = job_config)
top_df = q2.to_dataframe()
dict_q2 = top_df.set_index('state_fips_code')['COUNT'].to_dict()
dict_q2

{'48': 254, '13': 159, '51': 133, '21': 120, '29': 115}

In [5]:
#q3
price_per_tb = 6.25
tb = 1024 ** 4
cost_dict = {}
cost_dict['q1'] = q1.total_bytes_billed / tb * price_per_tb
cost_dict['q2'] = q2.total_bytes_billed / tb * price_per_tb
cost_dict

{'q1': 0.0010907649993896484, 'q2': 5.9604644775390625e-05}

In [6]:
#q4
ds = bigquery.Dataset("striking-audio-398801.p8")
bq.create_dataset(ds, exists_ok=True)
job_config_parq = bigquery.LoadJobConfig(source_format=bigquery.SourceFormat.PARQUET, write_disposition = "WRITE_TRUNCATE")
load_job = bq.load_table_from_uri(
    'gs://cs544_p8_2023/hdma-wi-2021.parquet',
    "striking-audio-398801.p8.HDMA", job_config = job_config_parq
)
[ds.dataset_id for ds in bq.list_datasets("striking-audio-398801")]

['CS544_p8_2023', 'P8', 'p8']

In [7]:
#q5
q5 = bq.query(
"""
select c.county_name, COUNT(*) as loan_applications
from `striking-audio-398801.p8.HDMA` as h
join `bigquery-public-data.geo_us_boundaries.counties` as c
on h.county_code = c.county_fips_code
group by c.county_name
order by loan_applications DESC
limit 10
""")
q5_df = q5.to_dataframe()
q5_dict = q5_df.set_index('county_name')['loan_applications'].to_dict()
q5_dict

{'Milwaukee': 46570,
 'Dane': 38557,
 'Waukesha': 34159,
 'Brown': 15615,
 'Racine': 13007,
 'Outagamie': 11523,
 'Kenosha': 10744,
 'Washington': 10726,
 'Rock': 9834,
 'Winnebago': 9310}

In [8]:
url = "https://docs.google.com/spreadsheets/d/11UeIBqQylAyNUBsIO54p6WiYJWHayQMfHDbUWq1jGco/"

external_config = bigquery.ExternalConfig("GOOGLE_SHEETS")
external_config.source_uris = [url]
external_config.options.skip_leading_rows = 1
external_config.autodetect = True

table = bigquery.Table('striking-audio-398801.p8.applications')
table.external_data_configuration = external_config

table = bq.create_table(table, exists_ok=True)

In [9]:
#q6
q6 = bq.query("""
select *
from striking-audio-398801.p8.applications
where income = 200000
""")
q6_df = q6.to_dataframe()
len(q6_df)

20

In [10]:
#q7
q7 = bq.query("""
select c.county_name, COUNT(*) as application_count
from striking-audio-398801.p8.applications as g
join bigquery-public-data.geo_us_boundaries.counties as c
on ST_CONTAINS(c.county_geom, ST_GEOGPOINT(g.longitude, g.latitude))
where c.state_fips_code = '55'
group by c.county_name
""")
q7_df = q7.to_dataframe()
q7_dict = q7_df.set_index('county_name')['application_count'].to_dict()
q7_dict

{'Door': 18,
 'Wood': 2,
 'Dane': 59,
 'Sauk': 2,
 'Winnebago': 3,
 'Walworth': 4,
 'Oneida': 3,
 'Brown': 6,
 'Dodge': 1,
 'Marathon': 1,
 'Polk': 1,
 'Milwaukee': 4,
 'Green Lake': 2,
 'Bayfield': 5,
 'Ozaukee': 1,
 'Jefferson': 2,
 'Sheboygan': 4,
 'Marinette': 2,
 'Eau Claire': 1,
 'Monroe': 2,
 'Douglas': 1,
 'Ashland': 2,
 'Manitowoc': 1,
 'Columbia': 4,
 'Crawford': 1,
 'Barron': 2,
 'Iowa': 1,
 'Kewaunee': 2,
 'Adams': 1,
 'St. Croix': 1,
 'Outagamie': 1,
 'Chippewa': 1}

In [11]:
import time
while True:
    models = list(bq.list_models('striking-audio-398801.p8')) 
    if any(model.model_id == 'lr_p8' for model in models): 
        break
    time.sleep(5)

In [12]:
#q8
bq.query("""
CREATE OR REPLACE MODEL `striking-audio-398801.p8.lr_p8`
OPTIONS(model_type='LINEAR_REG', input_label_cols=['loan_amount']) AS
SELECT loan_amount, income, loan_term
FROM
  `striking-audio-398801.p8.HDMA`
""")

eval_q8 = bq.query("""
SELECT
  *
FROM
  ML.EVALUATE(MODEL `striking-audio-398801.p8.lr_p8`,
  (
    SELECT
      loan_amount,
      income,
      loan_term
    FROM
      `striking-audio-398801.p8.HDMA`
  ))
""").to_dataframe()

eval_q8['r2_score'][0]

0.291653469223921

In [13]:
#q9
q9_weight = bq.query("""
SELECT
  *
FROM
  ML.WEIGHTS(MODEL `striking-audio-398801.p8.lr_p8`)
WHERE
  processed_input = 'income'
""").to_dataframe()
q9_weight['weight'][0]

0.8046310253989896

In [14]:
#q10
practice_quer = bq.query("""
SELECT *
FROM ML.PREDICT(MODEL `striking-audio-398801.p8.lr_p8`, (
    SELECT loan_amount, income, 360 AS loan_term
    FROM `striking-audio-398801.p8.applications`
))
""")
q10_df = practice_quer.to_dataframe()
q10_count = 0
for i in range(len(q10_df)):
    if q10_df['loan_amount'][i] > q10_df['predicted_loan_amount'][i]:
        q10_count += 1
q10_count / len(q10_df)

0.592436974789916